Лабораторна робота №5
з дисципліни "Нейронні мережі"

студента групи АнД-41

Айрапетов Давид

Варіант № 1
Завдання
Виконати вирішення задачs класифікації для 3 класів з набору даних food101 з використанням різних моделей нейронних мереж:
1.1 CNN модель з лабораторної роботи 4;

1.2 Resnet модель;

1.3 Efficientnet модель (моделі 1.1-1.3 з використанням оптимізатора Adam);

1.4 Моделі 1.2,1.3 з використанням оптимізатора SGD.

Індекси класів визначити індивідуально за залежностями: i1=n-1,i2=n+29,i3=n+59 (де і1,і2,і3 - індекс класу (починаючи з 0) у відсортованому за алфавітом наборі даних, n - номер за списком групи.

Порівняти результати моделювання із використанням TensorBoard.

Графік(и) порівняння результатів завантажити у форматі .svg та вставити у підсумковий файл поряд із та відповідними висновками.

Результати оформити у вигляді файлу з кодом для моделювання та візуалізації у форматі .ipynb.

Результати викласти у репозиторії GitHub (папка Lab5).

Підготуватися до захисту лабораторрної роботи (пояснення коду, відповіді на запитання за темою).

Індекси за варіантом: 0,30,60

In [1]:
import pathlib
import numpy as np

# turn our training path into a Python path
data_dir = pathlib.Path("C:/Users/mrdip/Desktop/lab5/101_food/train/")

# created a list of class_names from the subdirectories
class_names = np.array(sorted([item.name for item in data_dir.glob('*')])) 
print(class_names)

['apple_pie' 'deviled_eggs' 'lobster_bisque']


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "C:/Users/mrdip/Desktop/lab5/101_food/test/"
test_dir = "C:/Users/mrdip/Desktop/lab5/101_food/train/"

In [3]:
# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="categorical", # type of problem we're working on
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

Found 750 images belonging to 3 classes.
Found 225 images belonging to 3 classes.


In [4]:
# Create tensorboard callback (functionized because need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [8]:
import tensorflow_hub as hub

In [9]:
resnet_url="https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/5"
efficientnet_url="https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [10]:
IMAGE_SHAPE=(224,224)

In [11]:
from tensorflow.keras.layers import Conv2D, MaxPool2D,Flatten,Dense

def create_BC_model(model_url):
  """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  
  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SHAPE+(3,)) # define the input image shape
  
  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    Dense(3, activation='softmax', name='output_layer') # create our own output layer      
  ])

  return model

In [12]:
resnet_model= create_BC_model(resnet_url)
resnet_model.compile(
    loss="categorical_crossentropy", 
    optimizer=tf.keras.optimizers.Adam(), 
    metrics=['accuracy']
)

In [13]:
resnet_model.fit(train_data, 
epochs=2, validation_data=valid_data, 
callbacks=[create_tensorboard_callback(dir_name="lab5", experiment_name="resnet")]
)

Saving TensorBoard log files to: lab5/resnet/20211209-062235
Epoch 1/2
24/24 [==============================] - 82s 3s/step - loss: 0.5827 - accuracy: 0.7493 - val_loss: 0.2821 - val_accuracy: 0.8844
Epoch 2/2
24/24 [==============================] - 68s 3s/step - loss: 0.1650 - accuracy: 0.9520 - val_loss: 0.2256 - val_accuracy: 0.9156


In [14]:
resnet_model.evaluate(valid_data)

8/8 [==============================] - 16s 2s/step - loss: 0.2256 - accuracy: 0.9156


[0.22560890018939972, 0.9155555367469788]

In [15]:
resnet_pred_1 = resnet_model.predict(valid_data).argmax(axis=1)
resnet_pred_1

array([1, 1, 1, 1, 1, 2, 0, 2, 2, 1, 2, 2, 2, 0, 0, 0, 2, 0, 1, 2, 2, 2,
       1, 1, 1, 2, 0, 2, 0, 0, 0, 2, 0, 1, 1, 1, 1, 0, 0, 1, 1, 2, 0, 0,
       2, 0, 0, 2, 1, 0, 1, 0, 0, 1, 1, 0, 2, 0, 1, 1, 1, 2, 0, 0, 2, 0,
       1, 1, 0, 0, 1, 1, 2, 2, 2, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 0, 2, 0,
       1, 2, 1, 0, 1, 2, 1, 0, 2, 0, 1, 2, 0, 2, 2, 2, 0, 1, 1, 2, 1, 0,
       1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 0, 0, 0, 2, 0, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 2, 0, 2, 1, 1, 2, 0, 1, 2, 2, 2, 0, 2, 1, 1, 1, 0,
       0, 1, 2, 0, 1, 2, 2, 0, 0, 1, 0, 1, 2, 1, 2, 1, 1, 0, 2, 1, 0, 2,
       2, 2, 2, 1, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 1,
       1, 2, 2, 0, 0, 2, 0, 0, 2, 1, 0, 2, 2, 1, 2, 2, 1, 0, 2, 0, 0, 1,
       0, 0, 1, 2, 0], dtype=int64)

In [16]:
true_y = []
for batch in range(len(valid_data)):
    true_y += list(valid_data[batch][-1].argmax(axis=1))

In [17]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [18]:
def conf_matr(y_pred, y_true):
    conf_matrix = pd.DataFrame(confusion_matrix(y_pred=y_pred, y_true=y_true), 
                            index=class_names, 
                            columns=class_names)
    return conf_matrix

In [19]:
conf_matr(resnet_pred_1, true_y)

,apple_pie,deviled_eggs,lobster_bisque
apple_pie,21,23,31
deviled_eggs,28,26,21
lobster_bisque,22,30,23


In [20]:
efficient_model= create_BC_model(efficientnet_url)
efficient_model.compile(
    loss="categorical_crossentropy", 
    optimizer=tf.keras.optimizers.Adam(), 
    metrics=['accuracy']
)

In [21]:
efficient_model.fit(train_data, 
epochs=2, validation_data=valid_data, 
callbacks=[create_tensorboard_callback(dir_name="lab5", experiment_name="efficient")]
)

Saving TensorBoard log files to: lab5/efficient/20211209-062550
Epoch 1/2
24/24 [==============================] - 37s 1s/step - loss: 0.5576 - accuracy: 0.8240 - val_loss: 0.3411 - val_accuracy: 0.9244
Epoch 2/2
24/24 [==============================] - 27s 1s/step - loss: 0.2152 - accuracy: 0.9573 - val_loss: 0.2371 - val_accuracy: 0.9333


In [22]:
efficient_pred_1 = efficient_model.predict(valid_data).argmax(axis=1)
efficient_pred_1

array([0, 2, 2, 1, 0, 2, 2, 1, 0, 2, 1, 1, 0, 2, 1, 2, 0, 2, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 2, 0, 2, 1, 0, 0,
       1, 2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 2, 2, 1, 1, 0, 2,
       2, 0, 1, 1, 2, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2,
       0, 0, 1, 2, 2, 2, 2, 0, 1, 0, 2, 0, 2, 0, 2, 1, 1, 0, 1, 2, 0, 1,
       2, 1, 1, 2, 0, 2, 0, 2, 1, 1, 1, 2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 1,
       1, 2, 2, 2, 0, 1, 0, 2, 0, 2, 2, 2, 1, 2, 1, 0, 0, 2, 1, 2, 0, 2,
       0, 1, 1, 2, 2, 0, 2, 0, 0, 2, 2, 1, 2, 2, 1, 0, 2, 1, 1, 0, 2, 1,
       2, 0, 1, 0, 0, 1, 2, 2, 0, 2, 0, 1, 2, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 2, 0, 2, 2, 1, 1, 1, 1, 0, 2, 1, 0, 1, 2, 0, 2, 2, 1, 1,
       2, 1, 1, 1, 2], dtype=int64)

In [23]:
conf_matr(efficient_pred_1, true_y)

,apple_pie,deviled_eggs,lobster_bisque
apple_pie,23,31,21
deviled_eggs,30,19,26
lobster_bisque,21,22,32
